In [ ]:
# Here we implement the K-means algorithm
class KMeans:
  """
  Simple K-means clustering algorithm implementation.
    
  Parameters
  ----------
  n_clusters : int, default=2
      The number of clusters to form.
  max_iter : int, default=300
      The maximum number of iterations to run the algorithm.
  tol: float, default=1e-4
      Relative tolerance with regards to Frobenius norm of the difference
      in the cluster centers of two consecutive iterations to declare
      convergence.
  random_state : int, default=None
      Determines random number generation for centroid initialization. Use
      an int to make the randomness deterministic.
        
  Attributes
  ----------
  labels_ : ndarray of shape (n_samples,)
      The predicted labels of data points.
   n_iter_: int
      Number of iterations run.
  cluster_centers_ : ndarray of shape (n_clusters, n_features)
      The coordinates of the final cluster centers.
  distance_: ndarray of shape (n_samples, n_clusters)
      The final squared distances between data points and cluster centers.
  inertia_ : float
      The sum of squared distances of the samples to their closest centroid.
  """

  def __init__(self, n_clusters=2, max_iter=300, tol=1e-4, random_state=None):
    self.n_clusters = n_clusters
    self.max_iter = max_iter
    self.tol = tol
    self.random_state = random_state
    
  def fit(self, X):
    # Initialize centroids randomly (we choose randomly n_clusters data points from our dataset)
    # We can initialize the centroids with numerous other ways (e.g. the initialization in 'k-means++' algorithm) (fixme: implement this and compare the results)
    # For better results we can run the algorithm multiple times with different random initializations and choose the one with the best objective value (fixme: implement this and compare the results)
    # Changes for time efficiency can also be done
    rng = np.random.default_rng(self.random_state)
    self.cluster_centers_ = X[rng.permutation(X.shape[0])[:self.n_clusters]]
    self.distance_ = np.zeros((X.shape[0], self.n_clusters))
        
    # Iterate until convergence or maximum number of iterations is reached
    for i in range(1, self.max_iter+1):
      # Calculate squared Euclidean distance between data points and cluster centers (fixme: we can use other norms as well and compare the results)
      for j in range(self.n_clusters):
        self.distance_[:, j] = np.sum((X - self.cluster_centers_[j]) ** 2, axis=1)

      # Classify data points to the nearest centroid
      self.labels_ = np.argmin(self.distance_, axis=1) # predicted labels
            
      # Compute new centroids as the mean of the assigned data points in each cluster
      new_centroids = np.array([X[self.labels_ == label].mean(axis=0) for label in range(self.n_clusters)])

      # Count the iterations of the algorithm so far
      self.n_iter_ = i

      # Calculate the objective function/inertia
      self.inertia_ = np.sum(self.distance_[np.arange(len(self.distance_)), self.labels_])
            
      # Check for convergence (with regards to Frobenius norm - we can use numerous other ways)
      error = np.linalg.norm(self.cluster_centers_ - new_centroids)
      if error < self.tol:
        break
      if i != self.max_iter: # do not update the centroids in last iteration
        self.cluster_centers_ = new_centroids # update the centroids if not converged yet
  
  def predict(self, X):
    # Assign data points to closest centroid
    distance = np.zeros((X.shape[0], self.n_clusters))
    for j in range(self.n_clusters):
      distance[:, j] = np.sum((X - self.cluster_centers_[j]) ** 2, axis=1)
    
    labels = np.argmin(distance, axis=1) # predicted labels
    return labels

  def fit_predict(self, X):
    self.fit(X)
    return self.predict(X)

  def transform(self, X):
    # Transform data to distance matrix (distance matrix contains the Euclidean distances between data points and cluster centers)
    distance = np.zeros((X.shape[0], self.n_clusters))
    for j in range(self.n_clusters):
      distance[:, j] = np.sqrt(np.sum((X - self.cluster_centers_[j]) ** 2, axis=1))
    return distance

  def fit_transform(self, X):
    self.fit(X)
    return self.transform(X)

  def score(self, X):
    # Calculate score (negative inertia)
    distance = np.zeros((X.shape[0], self.n_clusters))
    for j in range(self.n_clusters):
      distance[:, j] = np.sum((X - self.cluster_centers_[j]) ** 2, axis=1)
    labels = np.argmin(distance, axis=1) # predicted labels
    return -np.sum(distance[np.arange(len(distance)), labels])